# Gravix Layer Cookbook: Text-to-SQL Multi-Agent System

Welcome to the Gravix Layer Cookbook series. This guide demonstrates building a multi-agent system that converts natural language questions into SQL queries and provides insightful analysis of the results, leveraging Gravix Layer's AI capabilities.

## Vision

Imagine asking questions about your data in plain English and receiving accurate, actionable insights instantly, without writing SQL or understanding database schemas. This system transforms natural language into precise database queries and meaningful results through intelligent agent collaboration.

## Solution

The system uses four specialized AI agents:
- **Database Context Analyst**: Understands data structure and relationships
- **SQL Query Specialist**: Converts natural language to SQL queries
- **Query Execution Specialist**: Safely executes queries with validation
- **Data Analysis Specialist**: Interprets results for human-readable insights

## What You'll Build

This recipe provides a production-ready text-to-SQL system:
- Multi-agent workflow with specialized AI components
- Secure SQLite database interaction
- Interactive query interface with formatted results
- Robust error handling and validation
- Sample dataset for testing

## Prerequisites

- Python 3.8+
- Basic SQL and database knowledge
- Gravix Layer API access

## 1. Setup

Install required libraries and configure the environment.

### Install Dependencies

In [1]:
!pip install "openai>=1.0.0" "pandas>=1.5.0" "python-dotenv>=0.19.0" "langchain" --quiet


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


### Import Libraries

In [2]:
import os
import sqlite3
import pandas as pd
from datetime import datetime
from typing import Dict, Optional
from openai import OpenAI
from dotenv import load_dotenv
import getpass
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI

print("Libraries imported successfully.")

Libraries imported successfully.


## 2. Configure Gravix Layer Client

Set up the Gravix Layer API client with secure API key handling.

In [3]:
#GET YOUR GRAVIXLAYER_API_KEY FROM https://platform.gravixlayer.com/
load_dotenv()
api_key = os.getenv("GRAVIXLAYER_API_KEY") or getpass.getpass("Enter Gravix Layer API key: ")
os.environ["GRAVIXLAYER_API_KEY"] = api_key

client = OpenAI(
    base_url="https://api.gravixlayer.com/v1/inference",
    api_key=api_key,
)

print("Gravix Layer client configured.")

Gravix Layer client configured.


## 3. Sample Dataset Creation

Create a sample customer sales database for testing.

In [4]:
data = {
    'customer_id': [1, 1, 2, 3, 3, 3, 4, 5, 5],
    'customer_name': ['Alice', 'Alice', 'Bob', 'Charlie', 'Charlie', 'Charlie', 'David', 'Eve', 'Eve'],
    'product_name': ['Laptop', 'Mouse', 'Desk Chair', 'Laptop', 'Keyboard', 'Monitor', 'Mouse', 'Tablet', 'Headphones'],
    'category': ['Electronics', 'Electronics', 'Furniture', 'Electronics', 'Electronics', 'Electronics', 'Electronics', 'Electronics', 'Electronics'],
    'price': [1200, 25, 150, 1250, 75, 300, 30, 800, 120],
    'quantity': [1, 2, 1, 1, 1, 2, 3, 1, 2],
    'order_date': ['2023-02-20', '2023-02-21', '2023-04-01', '2023-06-01', '2023-06-01', '2024-01-05', '2024-03-15', '2024-02-10', '2024-02-10']
}

df = pd.DataFrame(data)
df['total_amount'] = df['price'] * df['quantity']

db_filename = 'sales_data.db'
conn = sqlite3.connect(db_filename)
cursor = conn.cursor()
table_name = 'customer_sales_data'
df.to_sql(table_name, conn, if_exists='replace', index=False)

print("Database created with table 'customer_sales_data'.")

Database created with table 'customer_sales_data'.


## 4. Database Tools

Implement secure database interaction tools.

In [5]:
class DatabaseTools:
    def __init__(self, cursor, table_name):
        self.cursor = cursor
        self.table_name = table_name
    
    def get_context(self):
        try:
            self.cursor.execute(f"PRAGMA table_info({self.table_name});")
            columns = self.cursor.fetchall()
            schema_info = "Schema (Column Name: Data Type):\n"
            for col in columns:
                schema_info += f"  - {col[1]}: {col[2]}\n"
            
            self.cursor.execute(f"SELECT * FROM {self.table_name} LIMIT 5;")
            sample_rows = self.cursor.fetchall()
            headers = [desc[0] for desc in self.cursor.description]
            sample_data = "\nSample Data (First 5 Rows):\n" + f"{', '.join(headers)}\n"
            for row in sample_rows:
                sample_data += f"{', '.join(map(str, row))}\n"
            
            return f"Table '{self.table_name}':\n{schema_info}{sample_data}"
        except Exception as e:
            return f"Error getting context: {str(e)}"
    
    def execute_query(self, sql_query):
        try:
            if not sql_query.strip().upper().startswith('SELECT'):
                return "Error: Only SELECT queries permitted."
            self.cursor.execute(sql_query)
            results = self.cursor.fetchall()
            if not results:
                return "Query returned no results."
            
            headers = [desc[0] for desc in self.cursor.description]
            result_str = f"Columns: {', '.join(headers)}\n"
            for i, row in enumerate(results[:20]):
                result_str += f"Row {i+1}: {', '.join(map(str, row))}\n"
            if len(results) > 20:
                result_str += f"... and {len(results) - 20} more rows."
            return result_str
        except Exception as e:
            return f"Error executing query: {str(e)}\nQuery: {sql_query}"

db_tools = DatabaseTools(cursor, table_name)
print("Database tools initialized.")

Database tools initialized.


## 5. Multi-Agent System

Implement the four specialized AI agents.

### 5.1 Database Context Analyst

In [6]:
class DatabaseContextAgent:
    def __init__(self, llm, db_tools):
        self.db_tools = db_tools
        self.context_chain = LLMChain(
            llm=llm,
            prompt=ChatPromptTemplate.from_messages([
                ("system", "You are a database analyst providing actionable insights."),
                ("human", """Database Context:\n{raw_context}\n\nUser Question: {user_question}\n\nProvide:\n1. Relevant columns\n2. Important data relationships\n3. Potential challenges""")
            ])
        )
    
    def analyze_context(self, user_question):
        raw_context = self.db_tools.get_context()
        try:
            response = self.context_chain.invoke({"raw_context": raw_context, "user_question": user_question})
            analysis = response["text"]
            return {"raw_context": raw_context, "analysis": analysis}
        except Exception as e:
            return {"raw_context": raw_context, "analysis": "Error in context analysis: " + str(e)}

### 5.2 SQL Query Specialist

In [7]:
class SQLQueryAgent:
    def __init__(self, llm):
        self.sql_chain = LLMChain(
            llm=llm,
            prompt=ChatPromptTemplate.from_messages([
                ("system", "You are a SQL expert. Return only executable SQL code."),
                ("human", """Database Context:\n{raw_context}\n\nContext Analysis:\n{analysis}\n\nUser Question: {user_question}\n\nWrite a precise SQL query to answer the question. Return only SQL code.""")
            ])
        )
    
    def generate_query(self, user_question, context_data):
        try:
            response = self.sql_chain.invoke({
                "raw_context": context_data["raw_context"],
                "analysis": context_data["analysis"],
                "user_question": user_question
            })
            sql_query = response["text"].strip().replace("```sql", "").replace("```", "")
            return {"sql_query": sql_query}
        except Exception as e:
            return {"sql_query": None, "error": str(e)}

### 5.3 Query Execution Specialist

In [8]:
class QueryExecutionAgent:
    def __init__(self, db_tools):
        self.db_tools = db_tools
    
    def execute_query(self, query_data):
        if query_data.get('error'):
            return {"error": f"Cannot execute query: {query_data['error']}"}
        sql_query = query_data['sql_query']
        if not sql_query:
            return {"error": "No SQL query provided."}
        results = self.db_tools.execute_query(sql_query)
        return {"results": results, "sql_query": sql_query}

### 5.4 Data Analysis Specialist

In [9]:
class ResultAnalysisAgent:
    def __init__(self, llm):
        self.analysis_chain = LLMChain(
            llm=llm,
            prompt=ChatPromptTemplate.from_messages([
                ("system", "You are a data analyst providing clear insights."),
                ("human", """Question: {user_question}\n\nSQL Query: {sql_query}\n\nResults: {results}\n\nProvide a clear, human-readable answer that:\n1. Answers the question\n2. Highlights key insights\n3. Uses specific numbers\n4. Is accessible to non-technical users""")
            ])
        )
    
    def analyze_results(self, user_question, execution_data):
        if execution_data.get('error'):
            return {"analysis": f"Analysis failed: {execution_data['error']}"}
        try:
            response = self.analysis_chain.invoke({
                "user_question": user_question,
                "sql_query": execution_data['sql_query'],
                "results": execution_data['results']
            })
            return {"analysis": response["text"]}
        except Exception as e:
            return {"analysis": "Query executed, but analysis failed. Review raw results."}

## 6. Multi-Agent Coordinator

Orchestrate the agents to process queries.

In [ ]:
class MultiAgentCoordinator:
    def __init__(self, db_tools):
        context_llm = ChatOpenAI(
            openai_api_key=os.environ["GRAVIXLAYER_API_KEY"],
            openai_api_base="https://api.gravixlayer.com/v1/inference",
            model="meta-llama/llama-3.1-8b-instruct",
            temperature=0.2,
            max_tokens=200
        )
        sql_llm = ChatOpenAI(
            openai_api_key=os.environ["GRAVIXLAYER_API_KEY"],
            openai_api_base="https://api.gravixlayer.com/v1/inference",
            model="meta-llama/llama-3.1-8b-instruct",
            temperature=0.1,
            max_tokens=200
        )
        analysis_llm = ChatOpenAI(
            openai_api_key=os.environ["GRAVIXLAYER_API_KEY"],
            openai_api_base="https://api.gravixlayer.com/v1/inference",
            model="meta-llama/llama-3.1-8b-instruct",
            temperature=0.3,
            max_tokens=300
        )
        self.context_agent = DatabaseContextAgent(context_llm, db_tools)
        self.sql_agent = SQLQueryAgent(sql_llm)
        self.execution_agent = QueryExecutionAgent(db_tools)
        self.analysis_agent = ResultAnalysisAgent(analysis_llm)
        print("Multi-agent system initialized.")
    
    def process_query(self, user_question):
        start_time = datetime.now()
        context_data = self.context_agent.analyze_context(user_question)
        query_data = self.sql_agent.generate_query(user_question, context_data)
        execution_data = self.execution_agent.execute_query(query_data)
        analysis_data = self.analysis_agent.analyze_results(user_question, execution_data)
        processing_time = (datetime.now() - start_time).total_seconds()
        
        return {
            "success": not execution_data.get("error"),
            "question": user_question,
            "context_analysis": context_data.get("analysis", ""),
            "sql_query": execution_data.get("sql_query", query_data.get("sql_query", "")),
            "results": execution_data.get("results", ""),
            "analysis": analysis_data.get("analysis", ""),
            "processing_time": processing_time
        }

## 7. Query Interface

Provide a function to process queries and display results.

In [11]:
def text_to_sql_query(user_question, db_tools):
    coordinator = MultiAgentCoordinator(db_tools)
    return coordinator.process_query(user_question)

def display_results(result):
    print("\nQuery Results")
    print("-" * 40)
    print(f"Question: {result['question']}")
    print(f"Processing Time: {result.get('processing_time', 0):.2f} seconds")
    print("\nAnalysis:")
    print(result['analysis'])
    print("\nSQL Query:")
    print(result['sql_query'])
    print("\nRaw Results:")
    print(result['results'])

print("Text-to-SQL system ready.")

Text-to-SQL system ready.


## 8. Demonstration

Test the system with example queries.

In [12]:
print("Demonstration: Text-to-SQL Queries\n")

questions = [
    "How much has each customer spent in total?",
    "What is the average order value by product category?",
    "Show me all electronics purchases over $100",
    "Which customer bought the most expensive item?"
]

for question in questions:
    print(f"Query: {question}")
    result = text_to_sql_query(question, db_tools)
    display_results(result)
    print("\n" + "=" * 40)

Demonstration: Text-to-SQL Queries

Query: How much has each customer spent in total?
Multi-agent system initialized.


/var/folders/z2/5cm6cfms4qv436j8t8m9_tf00000gn/T/ipykernel_14416/650229414.py:3: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  context_llm = ChatOpenAI(
/var/folders/z2/5cm6cfms4qv436j8t8m9_tf00000gn/T/ipykernel_14416/1547614311.py:4: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  self.context_chain = LLMChain(



Query Results
----------------------------------------
Question: How much has each customer spent in total?
Processing Time: 18.22 seconds

Analysis:
Here's the answer:

**Total Spending by Each Customer**

Based on our analysis of customer sales data, here's how much each customer has spent in total:

* **Customer 1**: Spent a total of **$1250**
* **Customer 2**: Spent a total of **$150**
* **Customer 3**: Spent a total of **$1925**
* **Customer 4**: Spent a total of **$90**
* **Customer 5**: Spent a total of **$1040**

**Key Insights:**

* Customer 1 has spent the most, with a total of $1250.
* Customers 3 and 5 are among the biggest spenders, with totals of $1925 and $1040 respectively.
* Customers 2 and 4 have spent relatively little, with totals of just $150 and $90.

These insights can help businesses identify their most valuable customers and tailor marketing efforts to retain them.

SQL Query:

SELECT 
  customer_id,
  SUM(total_amount) AS total_spent
FROM 
  customer_sales_da

# Conclusion

### What We've Accomplished

This cookbook delivers a production-ready text-to-SQL system powered by Gravix Layer:
- **Multi-Agent Architecture**: Four specialized AI agents for accurate query processing
- **Secure Database Access**: SELECT-only queries with validation
- **Intelligent Analysis**: Context-aware query generation and human-readable insights
- **Robust Design**: Comprehensive error handling

### Key Features

- **Context-Aware**: Analyzes database structure for precise queries
- **Secure**: Restricts queries to SELECT statements
- **Accessible**: Provides clear insights for non-technical users
- **Extensible**: Easily adaptable for additional functionality


### Key Takeaways

- Agent specialization enhances query accuracy
- Security validation ensures safe database interactions
- Context analysis enables intelligent query generation
- Human-readable output makes data accessible

You've built a powerful text-to-SQL system with Gravix Layer, ready to transform natural language into actionable database insights.